In [1]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense 
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers


In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

In [4]:
voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [5]:
glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

In [6]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [7]:
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [8]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(4,input_shape=(1,5))(embedded_sequences)
out = layers.Dense(1,activation='relu')(x)

model = keras.Model(int_sequences_input,out)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         1369800   
_________________________________________________________________
lstm (LSTM)                  (None, 4)                 1680      
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 1,371,485
Trainable params: 1,685
Non-trainable params: 1,369,800
_________________________________________________________________


In [9]:
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [11]:
history = model.fit(x=x_train,y=y_train,batch_size=32,epochs=100)

Epoch 1/100
176/176 [==============================] - 12s 57ms/step - loss: 17.5236 - mean_squared_error: 17.5236 - mean_absolute_error: 4.0299 - mean_absolute_percentage_error: 98.7161
Epoch 2/100
176/176 [==============================] - 10s 58ms/step - loss: 16.3455 - mean_squared_error: 16.3455 - mean_absolute_error: 3.8807 - mean_absolute_percentage_error: 94.6812
Epoch 3/100
176/176 [==============================] - 10s 57ms/step - loss: 13.6137 - mean_squared_error: 13.6137 - mean_absolute_error: 3.5072 - mean_absolute_percentage_error: 84.5520
Epoch 4/100
176/176 [==============================] - 10s 57ms/step - loss: 9.6592 - mean_squared_error: 9.6592 - mean_absolute_error: 2.8915 - mean_absolute_percentage_error: 67.9565
Epoch 5/100
176/176 [==============================] - 10s 56ms/step - loss: 6.9891 - mean_squared_error: 6.9891 - mean_absolute_error: 2.3919 - mean_absolute_percentage_error: 54.6873
Epoch 6/100
176/176 [==============================] - 10s 57ms/step 

In [12]:
mse = history.history['mean_squared_error']
mae = history.history['mean_absolute_error']
mpe = history.history['mean_absolute_percentage_error']

scores = pd.DataFrame()
scores["MSE"] = mse
scores["MAE"] = mae
scores["% Error"] = mpe

scores

,MSE,MAE,% Error
0,17.523640,4.029948,98.716141
1,16.345486,3.880728,94.681229
2,13.613725,3.507206,84.552032
3,9.659227,2.891476,67.956482
4,6.989066,2.391921,54.687313
...,...,...,...
95,1.281075,0.948409,28.368788
96,1.281146,0.948435,28.383373
97,1.281126,0.948778,28.356922
98,1.281077,0.948436,28.362108


In [14]:
scores = model.evaluate(x=x_val,y=y_val)
scores[1:]

44/44 [==============================] - 1s 11ms/step - loss: 1.2683 - mean_squared_error: 1.2683 - mean_absolute_error: 0.9375 - mean_absolute_percentage_error: 28.6094


[1.268326997756958, 0.9375067353248596, 28.609424591064453]